# ಪಠ್ಯ ವರ್ಗೀಕರಣ ಕಾರ್ಯ

ನಾವು ಉಲ್ಲೇಖಿಸಿದಂತೆ, ನಾವು ಸರಳ ಪಠ್ಯ ವರ್ಗೀಕರಣ ಕಾರ್ಯದ ಮೇಲೆ ಗಮನಹರಿಸುವೆವು, ಇದು **AG_NEWS** ಡೇಟಾಸೆಟ್ ಆಧಾರಿತವಾಗಿದೆ, ಇದು ಸುದ್ದಿಯ ಶೀರ್ಷಿಕೆಗಳನ್ನು 4 ವರ್ಗಗಳಲ್ಲಿ ಒಂದಾಗಿ ವರ್ಗೀಕರಿಸುವುದು: ವಿಶ್ವ, ಕ್ರೀಡೆ, ವ್ಯವಹಾರ ಮತ್ತು ವಿಜ್ಞಾನ/ತಂತ್ರಜ್ಞಾನ.

## ಡೇಟಾಸೆಟ್

ಈ ಡೇಟಾಸೆಟ್ [`torchtext`](https://github.com/pytorch/text) ಮೋಡ್ಯೂಲ್‌ನಲ್ಲಿ ನಿರ್ಮಿಸಲಾಗಿದೆ, ಆದ್ದರಿಂದ ನಾವು ಇದನ್ನು ಸುಲಭವಾಗಿ ಪ್ರವೇಶಿಸಬಹುದು.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

ಇಲ್ಲಿ, `train_dataset` ಮತ್ತು `test_dataset` ಎಂಬವುಗಳು ಕ್ರಮವಾಗಿ ಲೇಬಲ್ (ವರ್ಗದ ಸಂಖ್ಯೆ) ಮತ್ತು ಪಠ್ಯವನ್ನು ಹಿಂತಿರುಗಿಸುವ ಸಂಗ್ರಹಗಳನ್ನು ಹೊಂದಿವೆ, ಉದಾಹರಣೆಗೆ:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

ಹಾಗಾದರೆ, ನಮ್ಮ ಡೇಟಾಸೆಟ್‌ನ ಮೊದಲ 10 ಹೊಸ ಶೀರ್ಷಿಕೆಗಳನ್ನು ಮುದ್ರಣ ಮಾಡೋಣ:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

ಡೇಟಾಸೆಟ್‌ಗಳು ಇಟರೇಟರ್‌ಗಳಾಗಿರುವುದರಿಂದ, ನಾವು ಡೇಟಾವನ್ನು ಹಲವಾರು ಬಾರಿ ಬಳಸಬೇಕಾದರೆ ಅದನ್ನು ಲಿಸ್ಟ್‌ಗೆ ಪರಿವರ್ತಿಸಬೇಕಾಗುತ್ತದೆ:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## ಟೋಕನೈಜೆಷನ್

ಈಗ ನಮಗೆ ಪಠ್ಯವನ್ನು ಟೆನ್ಸರ್‌ಗಳಾಗಿ ಪ್ರತಿನಿಧಿಸಬಹುದಾದ **ಸಂಖ್ಯೆಗಳಾಗಿ** ಪರಿವರ್ತಿಸಬೇಕಾಗಿದೆ. ನಾವು ಪದಮಟ್ಟದ ಪ್ರತಿನಿಧಾನವನ್ನು ಬಯಸಿದರೆ, ನಾವು ಎರಡು ಕೆಲಸಗಳನ್ನು ಮಾಡಬೇಕಾಗುತ್ತದೆ:
* ಪಠ್ಯವನ್ನು **ಟೋಕನ್‌ಗಳಾಗಿ** ವಿಭಜಿಸಲು **ಟೋಕನೈಜರ್** ಅನ್ನು ಬಳಸುವುದು
* ಆ ಟೋಕನ್‌ಗಳ **ಶಬ್ದಕೋಶ**ವನ್ನು ರಚಿಸುವುದು.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

ಶಬ್ದಕೋಶವನ್ನು ಬಳಸಿಕೊಂಡು, ನಾವು ಸುಲಭವಾಗಿ ನಮ್ಮ ಟೋಕನೈಸ್ ಮಾಡಿದ ಸ್ಟ್ರಿಂಗ್ ಅನ್ನು ಸಂಖ್ಯೆಗಳ ಸೆಟ್ ಆಗಿ ಎನ್‌ಕೋಡ್ ಮಾಡಬಹುದು:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## ಪದಗಳ ಬ್ಯಾಗ್ ಪಠ್ಯ ಪ್ರತಿನಿಧಾನ

ಪದಗಳು ಅರ್ಥವನ್ನು ಪ್ರತಿನಿಧಿಸುವುದರಿಂದ, ಕೆಲವೊಮ್ಮೆ ನಾವು ವಾಕ್ಯದ ಕ್ರಮವನ್ನು ಪರಿಗಣಿಸದೆ, ಪ್ರತ್ಯೇಕ ಪದಗಳನ್ನು ನೋಡಿಕೊಂಡು ಪಠ್ಯದ ಅರ್ಥವನ್ನು ತಿಳಿದುಕೊಳ್ಳಬಹುದು. ಉದಾಹರಣೆಗೆ, ಸುದ್ದಿಗಳನ್ನು ವರ್ಗೀಕರಿಸುವಾಗ, *ಹವಾಮಾನ*, *ಹಿಮ* ಎಂಬ ಪದಗಳು *ಹವಾಮಾನ ಮುನ್ಸೂಚನೆ* ಅನ್ನು ಸೂಚಿಸುವ ಸಾಧ್ಯತೆ ಇರುತ್ತದೆ, ಆದರೆ *ಸ್ಟಾಕ್ಸ್*, *ಡಾಲರ್* ಎಂಬ ಪದಗಳು *ಆರ್ಥಿಕ ಸುದ್ದಿ* ಗೆ ಸೇರಬಹುದು.

**ಪದಗಳ ಬ್ಯಾಗ್** (BoW) ವೆಕ್ಟರ್ ಪ್ರತಿನಿಧಾನವು ಅತ್ಯಂತ ಸಾಮಾನ್ಯವಾಗಿ ಬಳಸುವ ಪರಂಪರাগত ವೆಕ್ಟರ್ ಪ್ರತಿನಿಧಾನವಾಗಿದೆ. ಪ್ರತಿ ಪದವು ಒಂದು ವೆಕ್ಟರ್ ಸೂಚ್ಯಂಕಕ್ಕೆ ಜೋಡಿಸಲಾಗುತ್ತದೆ, ಮತ್ತು ವೆಕ್ಟರ್ ಅಂಶವು ನೀಡಲಾದ ದಾಖಲೆಗಳಲ್ಲಿ ಆ ಪದದ ಸಂಭವನೆಯ ಸಂಖ್ಯೆಯನ್ನು ಹೊಂದಿರುತ್ತದೆ.

![ಪದಗಳ ಬ್ಯಾಗ್ ವೆಕ್ಟರ್ ಪ್ರತಿನಿಧಾನವನ್ನು ಮೆಮೊರಿಯಲ್ಲಿ ಹೇಗೆ ಪ್ರತಿನಿಧಿಸಲಾಗುತ್ತದೆ ಎಂಬುದನ್ನು ತೋರಿಸುವ ಚಿತ್ರ.](../../../../../translated_images/kn/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: BoW ಅನ್ನು ಪಠ್ಯದ ಪ್ರತ್ಯೇಕ ಪದಗಳ ಒನ್-ಹಾಟ್-ಎನ್‌ಕೋಡ್ ಮಾಡಿದ ವೆಕ್ಟರ್‌ಗಳ ಮೊತ್ತವೆಂದು ಕೂಡ ಪರಿಗಣಿಸಬಹುದು.

ಕೆಳಗಿನ ಉದಾಹರಣೆಯಲ್ಲಿ Scikit Learn ಪೈಥಾನ್ ಲೈಬ್ರರಿಯನ್ನು ಬಳಸಿ ಪದಗಳ ಬ್ಯಾಗ್ ಪ್ರತಿನಿಧಾನವನ್ನು ಹೇಗೆ ರಚಿಸುವುದು ಎಂಬುದನ್ನು ತೋರಿಸಲಾಗಿದೆ:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

ನಮ್ಮ AG_NEWS ಡೇಟಾಸೆಟ್‌ನ ವೆಕ್ಟರ್ ಪ್ರತಿನಿಧಿಯಿಂದ ಬ್ಯಾಗ್-ಆಫ್-ವರ್ಡ್ಸ್ ವೆಕ್ಟರ್ ಅನ್ನು ಲೆಕ್ಕಹಾಕಲು, ನಾವು ಕೆಳಗಿನ ಫಂಕ್ಷನ್ ಅನ್ನು ಬಳಸಬಹುದು:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **ಗಮನಿಸಿ:** ಇಲ್ಲಿ ನಾವು ಜಾಗತಿಕ `vocab_size` ಚರವನ್ನು ಬಳಸಿಕೊಂಡು ಶಬ್ದಕೋಶದ ಡೀಫಾಲ್ಟ್ ಗಾತ್ರವನ್ನು ನಿರ್ದಿಷ್ಟಪಡಿಸುತ್ತಿದ್ದೇವೆ. ಸಾಮಾನ್ಯವಾಗಿ ಶಬ್ದಕೋಶದ ಗಾತ್ರ ಬಹಳ ದೊಡ್ಡದಾಗಿರುವುದರಿಂದ, ನಾವು ಶಬ್ದಕೋಶದ ಗಾತ್ರವನ್ನು ಅತ್ಯಂತ ಸಾಮಾನ್ಯ ಪದಗಳಿಗೆ ಮಿತಿಗೊಳಿಸಬಹುದು. `vocab_size` ಮೌಲ್ಯವನ್ನು ಕಡಿಮೆ ಮಾಡಿ ಕೆಳಗಿನ ಕೋಡ್ ಅನ್ನು ಚಾಲನೆ ಮಾಡಿ, ಮತ್ತು ಅದು ನಿಖರತೆಯನ್ನು ಹೇಗೆ ಪ್ರಭಾವಿಸುತ್ತದೆ ಎಂದು ನೋಡಿ. ನೀವು ಕೆಲವು ನಿಖರತೆ ಇಳಿಕೆಯನ್ನು ನಿರೀಕ್ಷಿಸಬಹುದು, ಆದರೆ ನಾಟಕೀಯವಲ್ಲ, ಹೆಚ್ಚಿನ ಕಾರ್ಯಕ್ಷಮತೆಯ ಪರ್ಯಾಯವಾಗಿ.


## BoW ವರ್ಗೀಕರಣ ತರಬೇತಿ

ನಾವು ಈಗಾಗಲೇ ನಮ್ಮ ಪಠ್ಯದ Bag-of-Words ಪ್ರತಿನಿಧಾನವನ್ನು ಹೇಗೆ ನಿರ್ಮಿಸುವುದನ್ನು ಕಲಿತಿದ್ದೇವೆ, ಈಗ ಅದಕ್ಕೆ ಮೇಲ್ಭಾಗದಲ್ಲಿ ವರ್ಗೀಕರಣವನ್ನು ತರಬೇತಿಮಾಡೋಣ. ಮೊದಲು, ನಮ್ಮ ತರಬೇತಿ ಡೇಟಾಸೆಟ್ ಅನ್ನು ಇಂತಹ ರೀತಿಯಲ್ಲಿ ಪರಿವರ್ತಿಸಬೇಕಾಗುತ್ತದೆ, ಅಲ್ಲಿ ಎಲ್ಲಾ ಸ್ಥಾನಿಕ ವೆಕ್ಟರ್ ಪ್ರತಿನಿಧಾನಗಳನ್ನು bag-of-words ಪ್ರತಿನಿಧಾನಕ್ಕೆ ಪರಿವರ್ತಿಸಲಾಗುತ್ತದೆ. ಇದನ್ನು ಸಾಧಿಸಲು, `bowify` ಫಂಕ್ಷನ್ ಅನ್ನು `collate_fn` ಪರಾಮೀಟರ್ ಆಗಿ ಸಾಮಾನ್ಯ torch `DataLoader` ಗೆ ಪಾಸ್ ಮಾಡಬಹುದು:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

ಈಗ ನಾವು ಒಂದು ಸರಳ ವರ್ಗೀಕರಣ ನ್ಯೂರಲ್ ನೆಟ್‌ವರ್ಕ್ ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸೋಣ, ಇದು ಒಂದು ಲೀನಿಯರ್ ಲೇಯರ್ ಅನ್ನು ಹೊಂದಿದೆ. ಇನ್‌ಪುಟ್ ವೆಕ್ಟರ್‌ನ ಗಾತ್ರವು `vocab_size` ಗೆ ಸಮಾನವಾಗಿದ್ದು, ಔಟ್‌ಪುಟ್ ಗಾತ್ರವು ವರ್ಗಗಳ ಸಂಖ್ಯೆಗೆ (4) ಹೊಂದಿಕೆಯಾಗುತ್ತದೆ. ನಾವು ವರ್ಗೀಕರಣ ಕಾರ್ಯವನ್ನು ಪರಿಹರಿಸುತ್ತಿದ್ದೇವೆ ಆದ್ದರಿಂದ ಅಂತಿಮ ಸಕ್ರಿಯತೆ ಕಾರ್ಯವು `LogSoftmax()` ಆಗಿದೆ.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

ಈಗ ನಾವು ಸ್ಟ್ಯಾಂಡರ್ಡ್ PyTorch ತರಬೇತಿ ಲೂಪ್ ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುವೆವು. ನಮ್ಮ ಡೇಟಾಸೆಟ್ ಬಹಳ ದೊಡ್ಡದಾಗಿರುವುದರಿಂದ, ನಮ್ಮ ಬೋಧನಾ ಉದ್ದೇಶಕ್ಕಾಗಿ ನಾವು ಒಂದೇ epoch ಗೆ ಮಾತ್ರ ತರಬೇತಿ ಮಾಡುತ್ತೇವೆ, ಮತ್ತು ಕೆಲವೊಮ್ಮೆ ಒಂದು epoch ಕಿಂತ ಕಡಿಮೆ ಸಮಯಕ್ಕೂ (epoch_size ಪ್ಯಾರಾಮೀಟರ್ ಅನ್ನು ನಿರ್ದಿಷ್ಟಪಡಿಸುವುದರಿಂದ ನಾವು ತರಬೇತಿಯನ್ನು ಮಿತಿಗೊಳಿಸಬಹುದು). ತರಬೇತಿ ಸಮಯದಲ್ಲಿ ಸಂಗ್ರಹಿತ ತರಬೇತಿ ನಿಖರತೆಯನ್ನು ಕೂಡ ವರದಿ ಮಾಡುತ್ತೇವೆ; ವರದಿ ಮಾಡುವ ಆವರ್ತನೆಯನ್ನು report_freq ಪ್ಯಾರಾಮೀಟರ್ ಬಳಸಿ ನಿರ್ದಿಷ್ಟಪಡಿಸಲಾಗುತ್ತದೆ.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## ಬಿಗ್ರಾಮ್, ಟ್ರೈಗ್ರಾಮ್ ಮತ್ತು ಎನ್-ಗ್ರಾಮ್‌ಗಳು

ಬ್ಯಾಗ್ ಆಫ್ ವರ್ಡ್ಸ್ ವಿಧಾನದಲ್ಲಿ ಒಂದು ಮಿತಿ ಇದೆ ಎಂದರೆ ಕೆಲವು ಪದಗಳು ಬಹುಪದ ಅಭಿವ್ಯಕ್ತಿಗಳ ಭಾಗವಾಗಿರುತ್ತವೆ, ಉದಾಹರಣೆಗೆ, 'ಹಾಟ್ ಡಾಗ್' ಎಂಬ ಪದವು 'ಹಾಟ್' ಮತ್ತು 'ಡಾಗ್' ಎಂಬ ಪದಗಳಿಗಿಂತ ಸಂಪೂರ್ಣ ವಿಭಿನ್ನ ಅರ್ಥ ಹೊಂದಿದೆ. ನಾವು 'ಹಾಟ್' ಮತ್ತು 'ಡಾಗ್' ಪದಗಳನ್ನು ಯಾವಾಗಲೂ ಒಂದೇ ವೆಕ್ಟರ್‌ಗಳಿಂದ ಪ್ರತಿನಿಧಿಸಿದರೆ, ಅದು ನಮ್ಮ ಮಾದರಿಯನ್ನು ಗೊಂದಲಕ್ಕೆ ಒಳಪಡಿಸಬಹುದು.

ಇದನ್ನು ಪರಿಹರಿಸಲು, **ಎನ್-ಗ್ರಾಮ್ ಪ್ರತಿನಿಧಾನಗಳು** ಡಾಕ್ಯುಮೆಂಟ್ ವರ್ಗೀಕರಣ ವಿಧಾನಗಳಲ್ಲಿ ಸಾಮಾನ್ಯವಾಗಿ ಬಳಸಲಾಗುತ್ತವೆ, ಇಲ್ಲಿ ಪ್ರತಿ ಪದ, ದ್ವಿಪದ ಅಥವಾ ತ್ರಿಪದದ ಆವರ್ತನೆ ತರಬೇತಿ ವರ್ಗೀಕರಿಸುವವರಿಗೆ ಉಪಯುಕ್ತ ಲಕ್ಷಣವಾಗಿರುತ್ತದೆ. ಉದಾಹರಣೆಗೆ, ಬಿಗ್ರಾಮ್ ಪ್ರತಿನಿಧಾನದಲ್ಲಿ, ಮೂಲ ಪದಗಳ ಜೊತೆಗೆ ಎಲ್ಲಾ ಪದ ಜೋಡಿಗಳನ್ನು ಶಬ್ದಕೋಶಕ್ಕೆ ಸೇರಿಸಲಾಗುತ್ತದೆ.

ಕೆಳಗಿನ ಉದಾಹರಣೆಯಲ್ಲಿ Scikit Learn ಬಳಸಿ ಬಿಗ್ರಾಮ್ ಬ್ಯಾಗ್ ಆಫ್ ವರ್ಡ್ ಪ್ರತಿನಿಧಾನವನ್ನು ಹೇಗೆ ರಚಿಸುವುದು ಎಂಬುದನ್ನು ತೋರಿಸಲಾಗಿದೆ:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-ಗ್ರಾಮ್ ವಿಧಾನದ ಪ್ರಮುಖ ದೋಷವೆಂದರೆ ಶಬ್ದಕೋಶದ ಗಾತ್ರವು ಅತ್ಯಂತ ವೇಗವಾಗಿ ವೃದ್ಧಿಯಾಗುತ್ತದೆ. ಪ್ರಾಯೋಗಿಕವಾಗಿ, ನಾವು N-ಗ್ರಾಮ್ ಪ್ರತಿನಿಧಾನವನ್ನು ಕೆಲವು ಆಯಾಮ ಕಡಿತ ತಂತ್ರಗಳೊಂದಿಗೆ, ಉದಾಹರಣೆಗೆ *ಎಂಬೆಡ್ಡಿಂಗ್ಸ್* ಜೊತೆಗೆ ಸಂಯೋಜಿಸಬೇಕಾಗುತ್ತದೆ, ಇದನ್ನು ನಾವು ಮುಂದಿನ ಘಟಕದಲ್ಲಿ ಚರ್ಚಿಸುವೆವು.

ನಮ್ಮ **AG News** ಡೇಟಾಸೆಟ್‌ನಲ್ಲಿ N-ಗ್ರಾಮ್ ಪ್ರತಿನಿಧಾನವನ್ನು ಬಳಸಲು, ನಾವು ವಿಶೇಷ ngram ಶಬ್ದಕೋಶವನ್ನು ನಿರ್ಮಿಸಬೇಕಾಗುತ್ತದೆ:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


ನಾವು ಮೇಲಿನ ಕೋಡ್ ಅನ್ನು ಬಳಸಿ ವರ್ಗೀಕರಣೆಯನ್ನು ತರಬೇತಿಮಾಡಬಹುದು, ಆದರೆ ಅದು ಬಹಳ ಮೆಮೊರಿ-ಅಸಮರ್ಥವಾಗಿರುತ್ತದೆ. ಮುಂದಿನ ಘಟಕದಲ್ಲಿ, ನಾವು ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳನ್ನು ಬಳಸಿ ಬಿಗ್ರಾಮ್ ವರ್ಗೀಕರಣೆಯನ್ನು ತರಬೇತಿಮಾಡುತ್ತೇವೆ.

> **Note:** ನೀವು ಪಠ್ಯದಲ್ಲಿ ನಿರ್ದಿಷ್ಟ ಸಂಖ್ಯೆಯಿಗಿಂತ ಹೆಚ್ಚು ಬಾರಿ ಸಂಭವಿಸುವ nಗ್ರಾಮ್‌ಗಳನ್ನು ಮಾತ್ರ ಉಳಿಸಬಹುದು. ಇದರಿಂದ ಅಪರೂಪದ ಬಿಗ್ರಾಮ್‌ಗಳು ಹೊರತುಪಡಿಸಲಾಗುತ್ತದೆ ಮತ್ತು ಆಯಾಮತೆ ಬಹಳಷ್ಟು ಕಡಿಮೆಯಾಗುತ್ತದೆ. ಇದನ್ನು ಮಾಡಲು, `min_freq` ಪರಿಮಾಣವನ್ನು ಹೆಚ್ಚು ಮೌಲ್ಯಕ್ಕೆ ಹೊಂದಿಸಿ, ಮತ್ತು ಶಬ್ದಕೋಶದ ಉದ್ದದ ಬದಲಾವಣೆಯನ್ನು ಗಮನಿಸಿ.


## ಪದದ ಆವರ್ತನೆ ವಿರುದ್ಧ ದಾಖಲೆ ಆವರ್ತನೆ TF-IDF

BoW ಪ್ರತಿನಿಧಾನದಲ್ಲಿ, ಪದಗಳ ಸಂಭವಗಳು ಪದದ ಸ್ವಭಾವವನ್ನು ಪರಿಗಣಿಸದೆ ಸಮಾನವಾಗಿ ತೂಕ ನೀಡಲಾಗುತ್ತದೆ. ಆದರೆ, *a*, *in* ಮುಂತಾದ ಸಾಮಾನ್ಯ ಪದಗಳು ವರ್ಗೀಕರಣಕ್ಕೆ ವಿಶೇಷ ಪದಗಳಿಗಿಂತ ಕಡಿಮೆ ಮಹತ್ವದವು ಎಂಬುದು ಸ್ಪಷ್ಟವಾಗಿದೆ. ವಾಸ್ತವದಲ್ಲಿ, ಬಹುತೇಕ NLP ಕಾರ್ಯಗಳಲ್ಲಿ ಕೆಲವು ಪದಗಳು ಇತರರಿಗಿಂತ ಹೆಚ್ಚು ಪ್ರಾಸಂಗಿಕವಾಗಿರುತ್ತವೆ.

**TF-IDF** ಎಂದರೆ **term frequency–inverse document frequency**. ಇದು bag of words ನ ಒಂದು ಬದಲಾವಣೆ, ಇಲ್ಲಿ ಒಂದು ದಾಖಲೆದಲ್ಲಿ ಪದದ ಕಾಣಿಕೆ 0/1 ಬೈನರಿ ಮೌಲ್ಯ ಬದಲಾಗಿ, ಪದದ ಸಂಭವದ ಆಧಾರದ ಮೇಲೆ ತೇಲುವ ಬಿಂದು ಮೌಲ್ಯವನ್ನು ಬಳಸಲಾಗುತ್ತದೆ.

ವಿವರವಾಗಿ, ದಾಖಲೆ $j$ ಯಲ್ಲಿನ ಪದ $i$ ಯ ತೂಕ $w_{ij}$ ಅನ್ನು ಹೀಗೆ ವ್ಯಾಖ್ಯಾನಿಸಲಾಗುತ್ತದೆ:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
ಇಲ್ಲಿ
* $tf_{ij}$ ಎಂದರೆ $j$ ನಲ್ಲಿ $i$ ಯ ಸಂಭವಗಳ ಸಂಖ್ಯೆ, ಅಂದರೆ ನಾವು ಹಿಂದಿನ BoW ಮೌಲ್ಯವನ್ನು ನೋಡಿದ್ದೇವೆ
* $N$ ಎಂದರೆ ಸಂಗ್ರಹದಲ್ಲಿನ ದಾಖಲೆಗಳ ಸಂಖ್ಯೆ
* $df_i$ ಎಂದರೆ ಸಂಪೂರ್ಣ ಸಂಗ್ರಹದಲ್ಲಿ ಪದ $i$ ಹೊಂದಿರುವ ದಾಖಲೆಗಳ ಸಂಖ್ಯೆ

TF-IDF ಮೌಲ್ಯ $w_{ij}$ ಒಂದು ಪದವು ದಾಖಲೆಗಳಲ್ಲಿ ಎಷ್ಟು ಬಾರಿ ಕಾಣಿಸುತ್ತದೆ ಎಂಬುದರ ಪ್ರಕಾರ ಹೆಚ್ಚಾಗುತ್ತದೆ ಮತ್ತು ಆ ಪದವನ್ನು ಹೊಂದಿರುವ ದಾಖಲೆಗಳ ಸಂಖ್ಯೆಯಿಂದ ಕಡಿಮೆ ಮಾಡಲಾಗುತ್ತದೆ, ಇದು ಕೆಲವು ಪದಗಳು ಇತರರಿಗಿಂತ ಹೆಚ್ಚು ಬಾರಿ ಕಾಣಿಸುವುದನ್ನು ಸರಿಹೊಂದಿಸಲು ಸಹಾಯ ಮಾಡುತ್ತದೆ. ಉದಾಹರಣೆಗೆ, ಒಂದು ಪದವು ಸಂಗ್ರಹದಲ್ಲಿನ *ಪ್ರತಿ* ದಾಖಲೆದಲ್ಲಿಯೂ ಕಾಣಿಸಿದರೆ, $df_i=N$, ಆಗ $w_{ij}=0$ ಆಗಿ ಆ ಪದಗಳನ್ನು ಸಂಪೂರ್ಣವಾಗಿ ನಿರ್ಲಕ್ಷಿಸಲಾಗುತ್ತದೆ.

ನೀವು ಸುಲಭವಾಗಿ Scikit Learn ಬಳಸಿ ಪಠ್ಯದ TF-IDF ವೆಕ್ಟರೀಕರಣವನ್ನು ರಚಿಸಬಹುದು:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## ಸಮಾರೋಪ

TF-IDF ಪ್ರತಿನಿಧಿಗಳು ವಿಭಿನ್ನ ಪದಗಳಿಗೆ ಆವರ್ತನ ತೂಕವನ್ನು ನೀಡಿದರೂ, ಅವು ಅರ್ಥ ಅಥವಾ ಕ್ರಮವನ್ನು ಪ್ರತಿನಿಧಿಸಲು ಸಾಧ್ಯವಿಲ್ಲ. 1935 ರಲ್ಲಿ ಪ್ರಸಿದ್ಧ ಭಾಷಾಶಾಸ್ತ್ರಜ್ಞ ಜೆ. ಆರ್. ಫರ್ಥ್ ಹೇಳಿದಂತೆ, "ಒಂದು ಪದದ ಸಂಪೂರ್ಣ ಅರ್ಥ ಯಾವಾಗಲೂ ಸಾಂದರ್ಭಿಕವಾಗಿರುತ್ತದೆ, ಮತ್ತು ಸಾಂದರ್ಭಿಕತೆಯ ಹೊರಗಿನ ಅರ್ಥ ಅಧ್ಯಯನವನ್ನು ಗಂಭೀರವಾಗಿ ತೆಗೆದುಕೊಳ್ಳಲಾಗುವುದಿಲ್ಲ." ನಾವು ಈ ಕೋರ್ಸ್‌ನಲ್ಲಿ ನಂತರ ಭಾಷಾ ಮಾದರೀಕರಣವನ್ನು ಬಳಸಿ ಪಠ್ಯದಿಂದ ಸಾಂದರ್ಭಿಕ ಮಾಹಿತಿಯನ್ನು ಹೇಗೆ ಹಿಡಿಯುವುದು ಎಂಬುದನ್ನು ಕಲಿಯುತ್ತೇವೆ.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕರಣ**:  
ಈ ದಸ್ತಾವೇಜು AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯಿಗಾಗಿ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ತಪ್ಪುಗಳು ಅಥವಾ ಅಸತ್ಯತೆಗಳು ಇರಬಹುದು ಎಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಪ್ರಮುಖ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವಿಕೆ ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
